In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [2]:
bucket_name = 'bankapplication1234567121212121'
session=boto3.session.Session()
my_region=session.region_name
print(my_region)

ap-south-1


In [3]:
s3 = boto3.resource('s3')
try:
	if my_region=='ap-south-1':
		s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': my_region})
		print("S3 Bucket created successfully")
except Exception as e:
	print("Error: ", e)

Error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [4]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = f's3://{bucket_name}//{prefix}/output'
print(output_path)

s3://bankapplication1234567121212121//xgboost-as-a-built-in-algo/output


In [5]:
s3_client = session.client('s3')
response = s3_client.list_buckets()
# Output the bucket names
# for my_bucket_object in response.objects.all():
for bucket in response['Buckets']:
      print({bucket["Name"]})

{'bankapplication1234567121212121'}


In [6]:
import pandas as pd
import urllib

In [7]:
try:
	urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
	print('Success: downloaded bank_clean.csv.')
except Exception as e:
	print("Data load error", e)

Success: downloaded bank_clean.csv.


In [8]:
try:
	model_data = pd.read_csv('./bank_clean.csv', index_col=0)
	print('Success: Data loaded into dataframe')
except Exception as e:
	print('Data loader', e)

Success: Data loaded into dataframe


In [9]:
import numpy as np

train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7*len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [10]:
# uploading data in s3 bucket
import os

# sagemaker documentation suggests to have dependent variable in the first column
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

In [12]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data=f's3://{bucket_name}/{prefix}/train', content_type='csv')

In [13]:
# test data
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_train = sagemaker.TrainingInput(s3_data=f's3://{bucket_name}/{prefix}/test', content_type='csv')